## Постановка задачи
Загрузим данные, приведем их к числовым, заполним пропуски, нормализуем данные и оптимизируем память.

Разделим выборку на обучающую/проверочную в соотношении 80/20.

Построим 4 модели логистической регрессии: для 8, 6 и остальных классов, для 2, 5 и остальных, для 1, 7 и остальных, и для 4 и 3 - по убыванию частоты значения. Будем использовать перекрестную проверку при принятии решения об оптимальном наборе столбцов.

Проведем предсказание и проверим качество через каппа-метрику.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

© ITtensive, 2020

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import preprocessing

### Загрузка данных

In [2]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


### Предобработка данных

In [3]:
data["Product_Info_2_1"] = data["Product_Info_2"].str.slice(0, 1)
data["Product_Info_2_2"] = pd.to_numeric(data["Product_Info_2"].str.slice(1, 2))
data.drop(labels=["Product_Info_2"], axis=1, inplace=True)
for l in data["Product_Info_2_1"].unique():
    data["Product_Info_2_1" + l] = data["Product_Info_2_1"].isin([l]).astype("int8")
data.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)
data.fillna(value=-1, inplace=True)

### Оптимизация памяти

In [4]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [5]:
data = reduce_mem_usage(data)
print (data.info())

Потребление памяти меньше на 49.49 Мб (минус 84.9 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 133 entries, Id to Product_Info_2_1B
dtypes: float16(18), int16(1), int32(1), int8(113)
memory usage: 8.8 MB
None


### Общий набор столбцов для расчета

In [6]:
columns_groups = ["Insurance_History", "InsurеdInfo", "Medical_Keyword",
                  "Family_Hist", "Medical_History", "Product_Info"]
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Предобработка данных

In [7]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data,
                                                     columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

### Разделение данных
Преобразуем выборки в отдельные наборы данных

In [8]:
data_train, data_test = train_test_split(data_transformed,
                                         test_size=0.2)
data_train = pd.DataFrame(data_train)
data_test = pd.DataFrame(data_test)
print (data_train.head())

              0         1         2         3         4         5         6  \
33363 -0.748444 -1.687657  0.820652  0.154055  0.611857 -0.169414 -1.159587   
34204  0.612962  0.759700 -1.148525  0.281905  0.611857 -0.169414 -1.159587   
41014 -0.866351 -0.707398 -1.299572 -0.660989  0.611857 -0.169414  0.862391   
25660  1.600085  0.759700  0.289513  1.416575  0.611857 -0.169414  0.862391   
34300  0.355214  0.759700  0.213990 -0.013748 -1.634368 -0.169414  0.862391   

              7         8         9  ...       109       110       111  \
33363  1.101046 -1.156735  1.130555  ... -0.083689  0.441621 -0.149284   
34204  1.101046 -1.156735  1.130555  ... -0.083689  0.441621 -0.149284   
41014 -1.013721  0.861233 -0.928723  ... -0.083689  0.441621 -0.149284   
25660 -1.013721  0.866008 -0.928723  ... -0.083689  0.441621 -0.149284   
34300 -1.013721  0.864934 -0.928723  ... -0.083689  0.441621 -0.149284   

            112       113       114       115       116       117  Response  
33

### Логистическая регрессия
В обучающих данных пометим все классы, кроме 6 и 8, как 0 - и проведем обучение по такому набору данных.

Затем в оставшихся данных (в которых класс не равен 6 или 8) заменим все классы, кроме 7 и 1, на 0 - и снова проведем обучение. И т.д. Получим иерархию классификаторов:
8/6/нет -> 7/1/нет -> 2/5/нет -> 4/3

In [9]:
def regression_model (columns, df):
    x = pd.DataFrame(df, columns=columns)
    model = LogisticRegression(max_iter=1000)
    model.fit(x, df["Response"])
    return model

def logistic_regression(columns, df_train):
    model = regression_model(columns, df_train)
    logr_grid = GridSearchCV(model, {}, cv=5, n_jobs=2,
                    scoring=make_scorer(cohen_kappa_score))
    x = pd.DataFrame(df_train, columns=columns)
    logr_grid.fit(x, df_train["Response"])
    return logr_grid.best_score_

### Оптимальный набор столбцов
Для каждого уровня иерархии это будет свой набор столбцов в исходных данных.

### Перекрестная проверка
Разбиваем обучающую выборку еще на k (часто 5) частей, на каждой части данных обучаем модель. Затем проверяем 1-ю, 2-ю, 3-ю, 4-ю части на 5; 1-ю, 2-ю, 3-ю, 5-ю части на 4 и т.д.

В итоге обучение пройдет весь набор данных, и каждая часть набора будет проверена на всех оставшихся (перекрестным образом).

In [10]:
def find_opt_columns (data_train):
    kappa_score_opt = 0
    columns_opt = []
    for col in columns_transformed:
        kappa_score = logistic_regression([col], data_train)
        if kappa_score > kappa_score_opt:
            columns_opt = [col]
            kappa_score_opt = kappa_score
    for col in columns_transformed:
        if col not in columns_opt:
            columns_opt.append(col)
            kappa_score = logistic_regression(columns_opt, data_train)
            if kappa_score < kappa_score_opt:
                columns_opt.pop()
            else:
                kappa_score_opt = kappa_score
    return columns_opt, kappa_score_opt

Будем последовательно "урезать" набор данных при расчете более глубоких моделей: после получения разделения на 8 и остальные отсечем все данные со значением 8, и т.д.

После каждого расчета модели будем вычислять значения в проверочной выборке. Проверочную выборку нулями заполнять не будем, иначе оценка будет считаться некорректно.

Набор разделений 6/8, 2/5, 1/7, 3/4 дает наибольшую точность

In [11]:
responses = [[6, 8], [2, 5], [1, 7], [3, 4]]
logr_models = [{}]*len(responses)
data_train_current = data_train.copy()
i = 0
for response in responses:
    m_train = data_train_current.copy()
    if response != [3,4]:
        m_train["Response"] = m_train["Response"].apply(lambda x:0 if x not in response else x)
    columns_opt, kappa_score_opt = find_opt_columns(m_train)
    print (i, kappa_score_opt, columns_opt)
    logr_models[i] = {
        "model": regression_model(columns_opt, m_train),
        "columns": columns_opt
    }
    if response != [3,4]:
        data_train_current = data_train_current[~data_train_current["Response"].isin(response)]
    i += 1

0 0.4177708439342368 [3, 0, 2, 4, 5, 6, 9, 10, 14, 15, 20, 22, 23, 24, 25, 26, 28, 31, 34, 35, 38, 43, 46, 47, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61, 63, 68, 69, 73, 79, 81, 82, 83, 84, 85, 87, 88, 90, 93, 94, 95, 96, 97, 99, 101, 102, 103, 104, 105, 108, 109, 110, 111, 116, 117]
1 0.1890318340814155 [14, 2, 3, 5, 6, 8, 9, 15, 17, 18, 19, 23, 25, 26, 27, 29, 32, 34, 41, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 59, 60, 61, 65, 68, 69, 72, 73, 74, 75, 79, 80, 82, 83, 84, 86, 87, 88, 92, 93, 94, 100, 104, 107, 108, 109, 111, 112, 115, 117]
2 0.533997292619593 [3, 1, 2, 4, 5, 6, 8, 9, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 27, 29, 31, 34, 35, 36, 37, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 51, 52, 53, 54, 55, 59, 60, 61, 62, 64, 65, 66, 68, 69, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 91, 92, 93, 94, 95, 96, 97, 99, 102, 103, 104, 105, 106, 108, 111, 113, 114, 116]
3 0.45241908515871676 [3, 2, 12, 14, 15, 18, 19, 20, 30, 35, 36, 38, 40, 42, 4

### Предсказание данных и оценка модели
Последовательно считаем предсказания для каждой классификации. После этого объединяем предсказание по иерархии.

In [12]:
def logr_hierarchy (x):
    for response in range(0, len(responses)):
        if x["target" + str(response)] > 0:
            x["target"] = x["target" + str(response)]
            break;
    return x

In [13]:
for response in range(0, len(responses)):
    model = logr_models[response]["model"]
    columns_opt = logr_models[response]["columns"]
    x = pd.DataFrame(data_test, columns=columns_opt)
    data_test["target" + str(response)] = model.predict(x)

In [14]:
data_test = data_test.apply(logr_hierarchy, axis=1,
                            result_type="expand")
print (data_test.head())

              0         1         2         3         4         5         6  \
34689 -0.630538 -1.444237 -2.056662  0.154055  0.611857 -0.169414 -1.159587   
23615 -0.160284  0.516280  0.365036 -0.471212 -1.634368 -0.169414  0.862391   
48438 -0.042377  0.266282  1.424839 -0.187544 -1.634368 -0.169414  0.862391   
46619 -1.805488 -1.444237  0.365036 -1.613872 -1.634368 -0.169414  0.862391   
4781   0.072787  1.246540  1.122746 -0.607052  0.611857 -0.169414  0.862391   

              7         8         9  ...       114       115       116  \
34689  1.101046 -1.156735  1.130555  ...  1.604350 -0.216001 -0.128866   
23615 -1.013721  0.870987 -0.928723  ... -0.623305 -0.216001 -0.128866   
48438 -1.013721  0.861569 -0.928723  ... -0.623305 -0.216001 -0.128866   
46619 -1.013721  0.861569 -0.928723  ... -0.623305 -0.216001 -0.128866   
4781  -1.013721  0.861099 -0.928723  ... -0.623305 -0.216001 -0.128866   

            117  Response  target0  target1  target2  target3  target  
34689 -0

Кластеризация дает 0.192, kNN(100) - 0.3, простая лог. регрессия - 0.512

In [15]:
print ("Логистическая регрессия, 4 уровня:",
      round(cohen_kappa_score(data_test["target"],
                data_test["Response"], weights="quadratic"), 3))

Логистическая регрессия, 4 уровня: 0.48


### Матрица неточностей

In [16]:
print (confusion_matrix(data_test["target"],
                data_test["Response"]))

[[ 436  344   27   12  202  426  139  148]
 [ 157  228    5    2   78   68    8    2]
 [  37   46   66   33   82  181   23    4]
 [  65   74   62  151   31  364   35   63]
 [  77  142   10    0  210   66   23   12]
 [  14   14    2   13    9   93   13   25]
 [ 224  297   22   14  368  700  837  485]
 [ 226  189    8   36  148  386  493 3122]]
